In [5]:
import json
import pandas as pd
import opentrons.execute
from string import ascii_uppercase

def custom_labware(file_name, location):
    protocol = opentrons.execute.get_protocol_api("2.12")
    with open(str(file_name)) as labware_file:
        labware_def = json.load(labware_file)
        return protocol.load_labware_from_definition(labware_def, int(location))


def coord_iter(last_letter="H", last_number=12):
    letter_sequence = ascii_uppercase[:ascii_uppercase.index(last_letter) + 1]
    number_sequence = [i for i in range(1, last_number + 1)]
    coord_grid = [[f"{letter}{number}" for number in number_sequence] for letter in letter_sequence]
    for row in coord_grid:
        for item in row:
            yield item


In [6]:
class Worker:
    
    def __init__(self, sheet_name, index_col=0, vol_col=12, last_letter="H", last_number=12):
        self.index_col, self.vol_col = index_col, vol_col
        self.raw_instructions = self._parse_excel(sheet_name)
        self.sequence = list(coord_iter(last_letter, last_number))
        self.commands = list(map(self._command, self.raw_instructions))
        
    
    def _command(self, sheet_entry: list):
        try:
            attempt_index = sheet_entry[self.index_col] - 1
            location = self.sequence[attempt_index]
        except IndexError:
            print("WARNING: There are more compounds than plate positions.")
            print(f"Command translation has stopped at index {attempt_index}.")
            raise StopIteration # cuts off mapping prematurely.
        return {
            "location": location,
            "volume": int(sheet_entry[self.vol_col])
        }
    
    def _parse_excel(self, sheet_name):
        print("Reading spreadsheet data...")
        xl_data = pd.read_excel(sheet_name, engine="openpyxl")
        raw_instructions = xl_data.values.tolist()
        #print(raw_instructions)
        print("Finished reading.")
        return raw_instructions

In [8]:
dir(pd)


['Categorical',
 'CategoricalDtype',
 'CategoricalIndex',
 'DataFrame',
 'DateOffset',
 'DatetimeIndex',
 'DatetimeTZDtype',
 'ExcelFile',
 'ExcelWriter',
 'Float64Index',
 'Grouper',
 'HDFStore',
 'Index',
 'IndexSlice',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'Int64Index',
 'Int8Dtype',
 'Interval',
 'IntervalDtype',
 'IntervalIndex',
 'MultiIndex',
 'NaT',
 'NamedAgg',
 'Period',
 'PeriodDtype',
 'PeriodIndex',
 'RangeIndex',
 'Series',
 'SparseArray',
 'SparseDataFrame',
 'SparseDtype',
 'SparseSeries',
 'Timedelta',
 'TimedeltaIndex',
 'Timestamp',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'UInt64Index',
 'UInt8Dtype',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__docformat__',
 '__file__',
 '__getattr__',
 '__git_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_config',
 '_hashtable',
 '_lib',
 '_libs',
 '_np_version_under1p14',
 '_np_version_under1p15',
 '_np_version_under1p16',
 '_np_version_under1p17',
 '_tslib

In [9]:
test = Worker("SSFL_data.xlsx", last_letter="D", last_number=6)
test.commands

Reading spreadsheet data...
Finished reading.
Command translation has stopped at index 24.


[{'location': 'A1', 'volume': 1995},
 {'location': 'A2', 'volume': 1924},
 {'location': 'A3', 'volume': 1907},
 {'location': 'A4', 'volume': 1830},
 {'location': 'A5', 'volume': 1823},
 {'location': 'A6', 'volume': 1612},
 {'location': 'B1', 'volume': 1569},
 {'location': 'B2', 'volume': 1534},
 {'location': 'B3', 'volume': 1533},
 {'location': 'B4', 'volume': 1513},
 {'location': 'B5', 'volume': 1512},
 {'location': 'B6', 'volume': 1372},
 {'location': 'C1', 'volume': 1308},
 {'location': 'C2', 'volume': 1298},
 {'location': 'C3', 'volume': 1094},
 {'location': 'C4', 'volume': 1019},
 {'location': 'C5', 'volume': 991},
 {'location': 'C6', 'volume': 977},
 {'location': 'D1', 'volume': 904},
 {'location': 'D2', 'volume': 897},
 {'location': 'D3', 'volume': 875},
 {'location': 'D4', 'volume': 867},
 {'location': 'D5', 'volume': 863},
 {'location': 'D6', 'volume': 839}]